# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Для этого рассмотрим несколько предположений:
* [Наличие зависимости между наличием детей и возвратом кредита в срок.](#the_first_question)
* [Наличие зависимости между семейным положением и возвратом кредита в срок.](#the_second_question)
* [Наличие зависимости между уровнем дохода и возвратом кредита в срок.](#the_third_question)
* [Влияние разных целей кредита на его возврат в срок.](#the_fourth_question)

На основе этих тезисов сделаем вывод о том, как выглядит наиболее надежный заёмщик.

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Изучение общей информации

In [34]:
import pandas as pd
from pymystem3 import Mystem

In [35]:
# Изучим первые 5 строк полученных данных.
clients = pd.read_csv('/datasets/data.csv').copy()

print('Первые 5 строк полученного массива данных:')
clients.head()

Первые 5 строк полученного массива данных:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [36]:
print('Последние 5 строк полученного массива данных:')
clients.tail()

Последние 5 строк полученного массива данных:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [37]:
print('Выведем произвольные 5 строк полученного массива данных:')
clients.sample(n=5, random_state=10)

Выведем произвольные 5 строк полученного массива данных:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1719,0,374049.572829,58,среднее,1,женат / замужем,0,F,пенсионер,0,190286.376012,покупка жилья для семьи
16620,1,-321.394320,39,высшее,0,гражданский брак,1,F,компаньон,0,439279.558662,образование
3584,0,NaN,58,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,получение дополнительного образования
15021,0,NaN,60,высшее,0,женат / замужем,0,F,пенсионер,1,NaN,автомобили
8097,0,-940.515480,29,Высшее,0,женат / замужем,0,M,сотрудник,0,239474.422339,операции с недвижимостью


In [38]:
# Выведем общую информацию о массиве.
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [39]:
clients.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


Из описания датасета можем видеть, что 
* Максимальное количество детей `20`, а минимальное `-1`. Эта ошибка могла быть вызвана опечаткой при заполнении массива данных.
* Так же видно, что большая часть дней трудового стажа имеет отрицательные величины, а максимальная величина в этой колонке - `401755.4 дней` - превышает продолжительность жизни.
* Минимальный возраст клиента - `0 лет`, чего быть не может.
* `2174` пропусков содержат столбцы с ежемесячным доходом и трудовым стажем.

# Вывод

**Общая информация о файле с данными:**

В каждой строке таблицы содержатся данные о клиенте. Часть колонок описывает самого клиента: возраст, образование, пол, семейное положение и количество детей. Остальные данные рассказывают о финансовом положении: трудовой стаж, тип занятости, ежемесячный доход и наличие задолженности.

**Описание записи данных:**

Всего в таблице 12 столбцов с различными типами данных, названия колонок записаны без нарушения хорошего стиля. Однако, в данных встречаются пропуски, так как количество значений в столбцах `days_employed` и `total_income` отличается от количества значений в остальных колонках. Помимо пропусков, большая часть дней трудового стажа имеет отрицательные значения, чего быть не должно.

## Предобработка данных

### Обработка пропусков

In [40]:
# Просмотрим количество пропусков в каждой колонке.
clients.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [41]:
# Найдем медианные значения для ежемесячного дохода и общего трудового стажа в каждой группе.
median_values = clients.pivot_table(index='income_type', values=['total_income', 'days_employed'], aggfunc='median')

# Объединим полученную сводную таблицу и исходные данные
clients_with_medians = clients.merge(median_values, on='income_type', how='left')

# Заполним пропуски в трудовом стаже и ежемесячном доходе данными из таблицы с медианными значениями.
clients['days_employed'] = clients['days_employed'].fillna(clients_with_medians['days_employed_y'])
clients['total_income'] = clients['total_income'].fillna(clients_with_medians['total_income_y'])

# Удалим промежуточный датафрейм, который больше не пригодится.
del clients_with_medians
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [42]:
# Меняем отрицательные значения трудового стажа на положительные.
clients['days_employed'] = clients['days_employed'].abs()

# Выведем первые 10 строк таблицы.
clients.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


# Вывод

После получения информации о таблице были обнаружены `2174` пропущенных значения в колонках `days_employed` и `total_income`. Это может быть связано с тем, что не все клиенты успели предоставить точные данные или справки о своей работе, которые возможно получить только через обращения в специализированные подразделения. 

**Изменения:**

1. Чтобы заполнить пропуски, были выбраны медианные значения количества рабочих дней и ежемесячного дохода в каждой группе, определенной по типу занятости. Такой выбор обусловлен тем, что среднее арифметическое в качестве замены пропусков может не подойти, так как в колонках могут присутствовать выбросы.


2. Так же были заменены отрицательные значения трудового стажа на положительные.

### Замена типа данных

In [43]:
# Применим метод astype() для замены вещественного типа данных на целочисленный
clients['total_income'] = clients['total_income'].astype('uint32')
clients['days_employed'] = clients['days_employed'].astype('uint32')
clients['education_id'] = clients['education_id'].astype('uint8')
clients['family_status_id'] = clients['family_status_id'].astype('uint8')

# Получим информацию о таблице.
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null uint32
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null uint8
family_status       21525 non-null object
family_status_id    21525 non-null uint8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null uint32
purpose             21525 non-null object
dtypes: int64(3), object(5), uint32(2), uint8(2)
memory usage: 1.5+ MB


# Вывод

Требовалось провести замену вещественного типа данных на целочисленный в столбцах `total_income` и `days_employed`. Для этого был выбран метод `astype()`, так как пропуски в перечисленных колонках были устранены еще на этапе их обработки.

### Обработка дубликатов

In [44]:
# Получим уникальные значения в столбце gender
clients['gender'].unique().tolist()

['F', 'M', 'XNA']

In [45]:
# Получим уникальные значения в столбце purpose
clients['purpose'].unique().tolist()

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

In [46]:
# Получим уникальные значения в столбце family_status
clients['family_status'].unique().tolist()

['женат / замужем',
 'гражданский брак',
 'вдовец / вдова',
 'в разводе',
 'Не женат / не замужем']

In [47]:
# Получим уникальные значения в столбце education
clients['education'].unique().tolist()

['высшее',
 'среднее',
 'Среднее',
 'СРЕДНЕЕ',
 'ВЫСШЕЕ',
 'неоконченное высшее',
 'начальное',
 'Высшее',
 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
 'Неоконченное высшее',
 'НАЧАЛЬНОЕ',
 'Начальное',
 'Ученая степень',
 'УЧЕНАЯ СТЕПЕНЬ',
 'ученая степень']

In [48]:
# Приводим всю колонку с уровнем образования к нижнему регистру.
clients['education'] = clients['education'].str.lower()

# Удаляем явные дубликаты.
clients = clients.drop_duplicates().reset_index(drop=True)

# Проверяем, что дубликатов не осталось.
clients.duplicated().sum()

0

# Вывод

**Обнаруженные дубликаты:**

В колонке с уровнем образования были обнаружены одинаковые названия, записаные символами разного регистра, из-за чего невозможно было сразу же удалить все явные дубликаты. 

Возможная причина их появления: клиенты невнимательно читали инструкции по заполнению анкет и поэтому по-разному заполняли данный пункт. 

**Метод поиска дубликатов:**

Сначала были просмотрены уникальные значения в столбцах, содержащих категориальные данные. После чего было выяснено, что одинаковые значения содержатся в колонке `education`.

**Изменения:**

1. Каждое значение в колонке `education` приведено к нижнему регистру методом `str.lower()`
2. Удалены все явные дубликаты из таблицы, количество оставшихся явных дубликатов: `0`

### Лемматизация

In [49]:
m = Mystem()

# Получим уникальные значения в столбце purpose
clients['purpose'].unique().tolist()

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

In [50]:
# Выделим ключевые слова.
main_purposes = ['жилье', 'автомобиль', 'свадьба', 'образование', 'недвижимость']

# Создадим функцию, посредством которой произведем лемматизацию
# и замену цели получения кредита на более обобщенную.
def lemmatize(purpose):
    lemma = m.lemmatize(purpose)
    for word in main_purposes:
        if word in lemma:
            lemma = word
    return lemma

# Применим функцию к колонке с изначальными целями получения кредита.
clients['main_purpose'] = clients['purpose'].apply(lemmatize)

# Выведем первые 5 строк массива.
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,main_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [51]:
# Количество клиентов с каждой целью:
clients['main_purpose'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: main_purpose, dtype: int64

In [52]:
# Так как жилье входит в категорию "недвижимость",
# то произведем замену.
clients.loc[clients['main_purpose'] == 'жилье', 'main_purpose'] = 'недвижимость'

# Количество клиентов с каждой целью:
clients['main_purpose'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: main_purpose, dtype: int64

# Вывод

Чтобы провести лемматизацию были выделены 5 ключевых слов, встречающихся среди всех целей получения кредита. Если одно из них было в изначальной цели, то она заменялась на более обобщенную группу целей.

Посредством лемматизации среди всех целей получения кредита удалось выделить 5 основных:
* Недвижимость
* Жилье
* Автомобиль
* Образование
* Свадьба

Но так как жилье это и есть недвижимость, то две эти категории были объединены. В итоге полученные группы целей:
* Недвижимость
* Автомобиль
* Образование
* Свадьба

### Категоризация данных

In [53]:
# Проведем категоризацию дохода по четырем группам. 
max_income = clients['total_income'].max()
median_income = clients['total_income'].median()
median_up_income = clients['total_income'].quantile(0.75)
median_down_income = clients['total_income'].quantile(0.25)
min_income = clients['total_income'].min()

print(min_income, median_down_income, median_income, median_up_income, max_income)

20667 107623.0 142594.0 195820.25 2265604


In [54]:
# Создадим функцию, которая будет определять группу уровня дохода.
def income_grouper(income):
        if min_income <= income <= median_down_income:
            return 1
        if median_down_income < income <= median_income:
            return 2
        if median_income < income <= median_up_income:
            return 3
        if median_up_income < income <= max_income:
            return 4

clients['income_category'] = clients['total_income'].apply(income_grouper)
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,main_purpose,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,4
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,2
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,3
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,4
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,3


In [55]:
# Посмотрим, сколько клиентов относится к каждой категории дохода.
clients['income_category'].value_counts()

2    5479
4    5364
1    5364
3    5247
Name: income_category, dtype: int64

# Вывод

В соответствии с ежемесячным доходом клиенты были распределены по четырем группам:
* К группе 1, где доход находится в интервале `[20667; 107623]`, относится `5364` клиентов
* К группе 2, где доход находится в интервале `(10723; 142594]`, относится `5479` клиентов
* К группе 3, где доход находится в интервале `(142594; 195820.25]`, относится `5257` клиентов
* К группе 4, где доход находится в интервале `(195820.25; 2265604]`, относится `5364` клиентов



## Исследование зависимостей

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?** <a name="the_first_question"></a>

In [56]:
# Выведем информацию о количестве клиентов на каждое возможное количество детей.
clients['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Скорее всего, здесь имел место человеческий фактор, и вместо значения -1 хотели записать 1, а вместо 20 -  2.

In [57]:
# Поменяем данные в таблице в соответствии с ошибками.
clients.loc[clients['children'] == 20, 'children'] = 2
clients.loc[clients['children'] == -1, 'children'] = 1

# Выведем сколько клиентов имеют каждое количество детей. 
clients['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Так как в вопросе неважно количество детей, а важно лишь их наличие, то создадим отдельную колонку, в которой определим, есть ли у клиента дети.

In [58]:
# Если дети есть - группа 1, в противном случае - группа 0.
def having_children(amount):
    if amount > 0:
        return 1
    return 0

clients['having_children_group'] = clients['children'].apply(having_children)
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,main_purpose,income_category,having_children_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,4,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,2,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,3,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,4,1
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,3,0


In [59]:
# Составим сводную таблицу по количеству детей и наличию задолженности.
clients_children_debt = clients.pivot_table(index='having_children_group', values='debt').round(4)

clients_children_debt

,debt
having_children_group,
0,0.0754
1,0.0921


# Вывод

Таким образом, существует зависимость между наличием детей и возвратом кредита в срок. 
Это можно утверждать исходя из того, что вероятность наличия задолженности при наличии детей у клиента составляет `9.2%`,
а при отсутствии детей - `7.5%`.

Как вариант, это связано с тем, что для людей с детьми первостепенная задача сделать так, чтобы дети ни в чем не нуждались, а уже потом выплачивать задолженность.

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?** <a name="the_second_question"></a>

In [60]:
# Составим сводную таблицу по семейному положению
# и количеству клиентов без задолженностей и с задолженностями.
clients_family_status_debt = clients.pivot_table(index = ['family_status'], columns = 'debt', 
                                                 values = 'income_type', aggfunc = 'count')

# Для каждой группы вычислим вероятность возврата кредита в срок.
clients_family_status_debt['probability'] = (clients_family_status_debt[1] / 
                                            (clients_family_status_debt[0] + clients_family_status_debt[1])).round(3)

clients_family_status_debt.sort_values(by='probability', ascending=False)

debt,0,1,probability
family_status,,,
Не женат / не замужем,2536,274,0.098
гражданский брак,3763,388,0.093
женат / замужем,11408,931,0.075
в разводе,1110,85,0.071
вдовец / вдова,896,63,0.066


# Вывод

Если клиент овдовел или в разводе, то вероятность задолженности меньше, чем у остальных групп. Самая большая вероятность невозврата кредита в срок у группы клиентов, которые не состоят в браке.
Полученные значения для каждой группы клиентов:
* Не женат / не замужем: вероятность задолженности - `9.8%`
* Гражданский брак: вероятность задолженности - `9.3%`
* Женат / замужем: вероятность задолженности - `7.5%`
* В разводе: вероятность задолженности - `7.1%`
* Вдовец / вдова: вероятность задолженности - `6.6%`

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?** <a name="the_thierd_question"></a>

In [61]:
# Составим сводную таблицу по категориям клиентов, в зависимости от их ежемесячного дохода, 
# и количеству клиентов без задолженностей и с задолженностями.
clients_total_income_debt = clients.pivot_table(index = ['income_category'], columns = 'debt', 
                                                 values = 'income_type', aggfunc = 'count')

# Для каждой группы вычислим вероятность возврата кредита в срок.
clients_total_income_debt['probability'] = (clients_total_income_debt[1] / 
                                            (clients_total_income_debt[0] + clients_total_income_debt[1])).round(3)

clients_total_income_debt.sort_values(by='probability', ascending=False)

debt,0,1,probability
income_category,,,
2,4996,483,0.088
3,4799,448,0.085
1,4937,427,0.080
4,4981,383,0.071


# **Вывод**

У клиентов из группы с наиболее высоким доходом меньше вероятность задолженности, чем у всех остальных. Однако у клиентов из следующих двух групп, зарабатывающих меньше, вероятность задолженности выше, чем у группы клиентов с наименьшим ежемесячным доходом.

Это может быть связано с тем, что клиенты с наименьшим доходом более грамотно распоряжаются финансами.

- **Как разные цели кредита влияют на его возврат в срок?** <a name="the_fourth_question"></a>

In [62]:
# Составим сводную таблицу по обобщенным целям клиентов, 
# и количеству клиентов без задолженностей и с задолженностями.
clients_main_purpose_debt = clients.pivot_table(index = ['main_purpose'], columns = 'debt', 
                                                 values = 'income_type', aggfunc = 'count')

# Для каждой группы вычислим вероятность возврата кредита в срок.
clients_main_purpose_debt['probability'] = (clients_main_purpose_debt[1] / 
                                            (clients_main_purpose_debt[0] + clients_main_purpose_debt[1])).round(3)

clients_main_purpose_debt.sort_values(by='probability', ascending=False)

debt,0,1,probability
main_purpose,,,
автомобиль,3903,403,0.094
образование,3643,370,0.092
свадьба,2138,186,0.080
недвижимость,10029,782,0.072


# Вывод

Самая высокая вероятность задолженности у тех клиентов, которые собрались покупать автомобиль - она составляет `9.4%`. 
Это может быть связано с тем, что после покупки автомобиля появляются дополнительные траты на его же содержание. 
Наименьшая вероятность задолженности у клиентов, стремящихся приобрести недвижимость - `7.2%`. Далее следуют группы клиентов, которые берут кредит на свадьбу, у них может появиться задолженность с вероятностью `8%`, и группа клиентов, которая оплачивает образование - `9.2%`

## Шаг 4. Общий вывод

Было проведено исследование по следующим тезисам:

* [Наличие зависимости между наличием детей и возвратом кредита в срок.](#the_first_question)
* [Наличие зависимости между семейным положением и возвратом кредита в срок.](#the_second_question)
* [Наличие зависимости между уровнем дохода и возвратом кредита в срок.](the_third_question)
* [Влияние разных целей кредита на его возврат в срок.](the_fourth_question)

На основ результатов можно сделать вывод, что 
* Клиенты `в разводе` или `вдовцы / вдовы` имеют наименьшие вероятности невозврата кредита в срок - `7.1%` и `6.6%` соответственно; Самые ненадежные клиенты же - `не имеющие официального подтверждения отношений`, у них вероятность задолженности - `9.8%`.
* Клиенты без детей будут иметь задолженность с шансом `7.5%`, против `9.2%` у клиентов с детьми.
* Что касаемо доходов, наименьшая вероятность задолженности будет у зарабатывающих `более 196 тыс.рублей` в месяц - она составляет `7.1%`, в то время как наибольший шанс неоплаты в срок у группы клиентов, зарабатывающих `от 143 тыс.рублей до 196 тыс.рублей` в месяц, он составляет `8.8%`
* Наименьшая вероятность задолженности будет у клиентов, которые берут кредит с целью приобретения недвижимости - `7.2%`, когда наибольший шанс невозврата кредита в срок у клиентов, которые собираются покупать автомобиль, - `9.4%`.

Таким образом, наиболее надежный заёмщик - это овдовевший клиент без детей, который берет кредит в целях приобретения недвижимости и зарабатывает от 196 тысяч рублей в месяц.

Самая же ненадежная категория заёмщиков - холостые клиенты с детьми, стремящиеся приобрести автомобиль и зарабатывающие от 143 тыс.рублей до 196 тыс.рублей в месяц.